# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa4a8eec520>
├── 'a' --> tensor([[ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765]])
└── 'x' --> <FastTreeValue 0x7fa4a8eec4c0>
    └── 'c' --> tensor([[ 0.1583, -0.8993,  0.7182, -0.3910],
                        [ 0.8752, -1.8361,  1.1016,  0.1668],
                        [-0.3019,  0.4488, -0.7287,  0.6512]])

In [4]:
t.a

tensor([[ 1.9223,  0.8296,  0.3704],
        [-1.8741, -1.5719, -0.8765]])

In [5]:
%timeit t.a

75 ns ± 1.22 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa4a8eec520>
├── 'a' --> tensor([[ 0.2447,  1.5870, -0.4561],
│                   [-0.6478, -0.6055,  0.6983]])
└── 'x' --> <FastTreeValue 0x7fa4a8eec4c0>
    └── 'c' --> tensor([[ 0.1583, -0.8993,  0.7182, -0.3910],
                        [ 0.8752, -1.8361,  1.1016,  0.1668],
                        [-0.3019,  0.4488, -0.7287,  0.6512]])

In [7]:
%timeit t.a = new_value

73.4 ns ± 1.75 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765]]),
    x: Batch(
           c: tensor([[ 0.1583, -0.8993,  0.7182, -0.3910],
                      [ 0.8752, -1.8361,  1.1016,  0.1668],
                      [-0.3019,  0.4488, -0.7287,  0.6512]]),
       ),
)

In [10]:
b.a

tensor([[ 1.9223,  0.8296,  0.3704],
        [-1.8741, -1.5719, -0.8765]])

In [11]:
%timeit b.a

67.7 ns ± 1.54 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.3812,  0.8124,  1.0280],
               [ 0.3369, -1.2972, -0.6501]]),
    x: Batch(
           c: tensor([[ 0.1583, -0.8993,  0.7182, -0.3910],
                      [ 0.8752, -1.8361,  1.1016,  0.1668],
                      [-0.3019,  0.4488, -0.7287,  0.6512]]),
       ),
)

In [13]:
%timeit b.a = new_value

642 ns ± 17.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.05 µs ± 19.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.9 µs ± 238 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

289 µs ± 13.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

281 µs ± 7.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa400e79790>
├── 'a' --> tensor([[[ 1.9223,  0.8296,  0.3704],
│                    [-1.8741, -1.5719, -0.8765]],
│           
│                   [[ 1.9223,  0.8296,  0.3704],
│                    [-1.8741, -1.5719, -0.8765]],
│           
│                   [[ 1.9223,  0.8296,  0.3704],
│                    [-1.8741, -1.5719, -0.8765]],
│           
│                   [[ 1.9223,  0.8296,  0.3704],
│                    [-1.8741, -1.5719, -0.8765]],
│           
│                   [[ 1.9223,  0.8296,  0.3704],
│                    [-1.8741, -1.5719, -0.8765]],
│           
│                   [[ 1.9223,  0.8296,  0.3704],
│                    [-1.8741, -1.5719, -0.8765]],
│           
│                   [[ 1.9223,  0.8296,  0.3704],
│                    [-1.8741, -1.5719, -0.8765]],
│           
│                   [[ 1.9223,  0.8296,  0.3704],
│                    [-1.8741, -1.5719, -0.8765]]])
└── 'x' --> <FastTreeValue 0x7fa400e794c0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

46.3 µs ± 930 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa400b40f70>
├── 'a' --> tensor([[ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765],
│                   [ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765],
│                   [ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765],
│                   [ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765],
│                   [ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765],
│                   [ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765],
│                   [ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765],
│                   [ 1.9223,  0.8296,  0.3704],
│                   [-1.8741, -1.5719, -0.8765]])
└── 'x' --> <FastTreeValue 0x7fa4843f8f10>
    └── 'c' --> tensor([[ 0.1583, -0.8993,  0.7182, -0.3910],
                        [ 0.8752, -1.8361,  1.1016,  0.1668],
                 

In [23]:
%timeit t_cat(trees)

42.1 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78.3 µs ± 1.22 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 1.9223,  0.8296,  0.3704],
                [-1.8741, -1.5719, -0.8765]],
       
               [[ 1.9223,  0.8296,  0.3704],
                [-1.8741, -1.5719, -0.8765]],
       
               [[ 1.9223,  0.8296,  0.3704],
                [-1.8741, -1.5719, -0.8765]],
       
               [[ 1.9223,  0.8296,  0.3704],
                [-1.8741, -1.5719, -0.8765]],
       
               [[ 1.9223,  0.8296,  0.3704],
                [-1.8741, -1.5719, -0.8765]],
       
               [[ 1.9223,  0.8296,  0.3704],
                [-1.8741, -1.5719, -0.8765]],
       
               [[ 1.9223,  0.8296,  0.3704],
                [-1.8741, -1.5719, -0.8765]],
       
               [[ 1.9223,  0.8296,  0.3704],
                [-1.8741, -1.5719, -0.8765]]]),
    x: Batch(
           c: tensor([[[ 0.1583, -0.8993,  0.7182, -0.3910],
                       [ 0.8752, -1.8361,  1.1016,  0.1668],
                       [-0.3019,  0.4488, -0.7287,  0.6512]],
         

In [26]:
%timeit Batch.stack(batches)

110 µs ± 2.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765],
               [ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765],
               [ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765],
               [ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765],
               [ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765],
               [ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765],
               [ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765],
               [ 1.9223,  0.8296,  0.3704],
               [-1.8741, -1.5719, -0.8765]]),
    x: Batch(
           c: tensor([[ 0.1583, -0.8993,  0.7182, -0.3910],
                      [ 0.8752, -1.8361,  1.1016,  0.1668],
                      [-0.3019,  0.4488, -0.7287,  0.6512],
                      [ 0.1583, -0.8993,  0.7182, -0.3910],
                      [ 0.8752, -

In [28]:
%timeit Batch.cat(batches)

191 µs ± 3.05 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

580 µs ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
